Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features. Use the data from 2005.

Create a query to get the list of films and a boolean indicating if it was rented last month (August 2005). This would be our target variable.

Read the data into a Pandas dataframe.

Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.

Create a logistic regression model to predict this variable from the cleaned data.
Evaluate the results.

In [1]:
# Importing modules & setting up pass
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()
import numpy as np
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

········


In [2]:
# How many times a movie was rented? sakila.film vs sakila.rental
# Which category of movies in most rented? sakila.category vs sakila.rental vs sakila.film_category
# Which rating of movies is most rented? sakila.rating vs sakila.rental
# Has the length of the movie some influence? sakila.film vs sakila.rental
# What about the language? sakila.language vs sakila.film
# Special-features? sakila.film
# Rental price/rate? sakila.film
# Rental duration? sakila.film

In [3]:
# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [4]:
sakila =  '''
SELECT f.film_id, COUNT(r.rental_id) rented_times,f.rental_duration, c.name as category, f.length,
f.rating, f.special_features, f.rental_rate, l.name AS language,
CASE
    WHEN r.rental_date BETWEEN '2005-05-01' AND '2005-05-31' THEN True
    ELSE False END AS may
FROM sakila.film f
Left JOIN inventory i
    ON f.film_id = i.film_id
JOIN sakila.rental r
    ON i.inventory_id = r.inventory_id
Join sakila.film_category fc
    On fc.film_id = f.film_id
Join sakila.category c
    On c.category_id = fc.category_id
Join sakila.language l
    On l.language_id = f.language_id
Where r.rental_date Between '2005-01-01' AND '2005-12-31'
GROUP BY film_id, rental_duration,category, f.length, f.rating, may, f.special_features,f.rental_rate, language; 
'''


sakila = pd.read_sql_query(sakila, engine)

In [5]:
sakila.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1585 entries, 0 to 1584
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   film_id           1585 non-null   int64  
 1   rented_times      1585 non-null   int64  
 2   rental_duration   1585 non-null   int64  
 3   category          1585 non-null   object 
 4   length            1585 non-null   int64  
 5   rating            1585 non-null   object 
 6   special_features  1585 non-null   object 
 7   rental_rate       1585 non-null   float64
 8   language          1585 non-null   object 
 9   may               1585 non-null   int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 124.0+ KB


One hot encoding / Get dummies / Whatever! the special_features...

In [6]:
sakila['special_features'].value_counts()

Trailers,Commentaries,Behind the Scenes                   132
Deleted Scenes,Behind the Scenes                          117
Trailers                                                  115
Commentaries,Behind the Scenes                            110
Trailers,Commentaries                                     109
Behind the Scenes                                         109
Commentaries,Deleted Scenes,Behind the Scenes             108
Commentaries                                              106
Trailers,Deleted Scenes                                   106
Commentaries,Deleted Scenes                               103
Trailers,Behind the Scenes                                103
Deleted Scenes                                            100
Trailers,Commentaries,Deleted Scenes                       95
Trailers,Commentaries,Deleted Scenes,Behind the Scenes     92
Trailers,Deleted Scenes,Behind the Scenes                  80
Name: special_features, dtype: int64

In [7]:
#this can be improved...this is a lot of code to a simple task....
trailer = []; b_scenes = []; commentaries = []; s_features = []

for i in sakila['special_features']:
        if re.findall('Trailers', i):
            trailer.append(1)
        else:
            trailer.append(0)
for i in sakila['special_features']:
        if re.findall('Behind the Scenes', i):
            b_scenes.append(1)
        else:
            b_scenes.append(0)      
for i in sakila['special_features']:
        if re.findall('Commentaries', i):
            commentaries.append(1)
        else:
            commentaries.append(0)     
for i in sakila['special_features']:
        if re.findall('Deleted Scenes', i):
            s_features.append(1)
        else:
            s_features.append(0)
            
sakila['trailers'] = trailer
sakila['commentaries'] = commentaries
sakila['behind the scenes'] = b_scenes  
sakila['deleted scenes'] = s_features

In [8]:
sakila = sakila.drop(['special_features'], axis=1)
sakila = sakila.drop(['film_id'], axis=1)
#there's no need for it now

In [9]:
# as objects
sakila['rental_duration'] = sakila['rental_duration'].astype(object)
sakila['rental_rate'] = sakila['rental_rate'].astype(object)
sakila['language'] = sakila['language'].astype(object)
# as bools
sakila['trailers'] = sakila['trailers'].astype(bool)
sakila['may'] = sakila['may'].astype(bool)
sakila['behind the scenes'] = sakila['behind the scenes'].astype(bool)
sakila['commentaries'] = sakila['commentaries'].astype(bool)
sakila['deleted scenes'] = sakila['deleted scenes'].astype(bool)

In [10]:
y = sakila['may']
X = sakila.drop('may', axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
X_train_num = X_train.select_dtypes(include = np.number)
# Scaling data
transformer = MinMaxScaler().fit(X_train_num)
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized)

In [12]:
X_train_norm

,0,1
0,0.000000,0.330935
1,0.566667,0.863309
2,0.000000,0.482014
3,0.800000,0.784173
4,0.033333,0.079137
...,...,...
1263,0.166667,0.784173
1264,0.400000,0.410072
1265,0.233333,0.935252
1266,0.000000,0.223022


In [13]:
X_train_cat = X_train.select_dtypes(include = object)

In [14]:
X_train_categorical = X_train.select_dtypes(include = object)
X_train_cat = pd.get_dummies(X_train_categorical, 
                             columns=['category', 'rating','rental_duration', 'rental_rate', 'language'],
                             drop_first=True)

/Users/miguel/opt/anaconda3/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [15]:
X_train_transformed = np.concatenate([X_train_norm, X_train_cat], axis=1)

In [16]:
# now the tricky part! max number of iterations always reached with saga or lbfgs
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='newton-cg',
                  multi_class='multinomial').fit(X_train_transformed, y_train)

In [17]:
# now the X-test
X_test_num = X_test.select_dtypes(include = np.number)

# We'll use the transformer we used before while training data
X_test_normalized = transformer.transform(X_test_num)
X_test_norm = pd.DataFrame(X_test_normalized)

In [18]:
# for categoricals
X_test_categorical = X_test.select_dtypes(include = object)
X_test_cat = pd.get_dummies(X_test_categorical, 
                            columns=['category', 'rating','rental_duration', 'rental_rate', 'language'],
                            drop_first=True)
# verify that dummies columns are in the same order and that the same column was dropped
display(list(zip(list(X_train_cat.columns),list(X_test_cat.columns))))
# not needed if you treat each dataframe with one_hot_encoder and save the encode (and the column names)
X_test_cat.head()

/Users/miguel/opt/anaconda3/lib/python3.9/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


[('category_Animation', 'category_Animation'),
 ('category_Children', 'category_Children'),
 ('category_Classics', 'category_Classics'),
 ('category_Comedy', 'category_Comedy'),
 ('category_Documentary', 'category_Documentary'),
 ('category_Drama', 'category_Drama'),
 ('category_Family', 'category_Family'),
 ('category_Foreign', 'category_Foreign'),
 ('category_Games', 'category_Games'),
 ('category_Horror', 'category_Horror'),
 ('category_Music', 'category_Music'),
 ('category_New', 'category_New'),
 ('category_Sci-Fi', 'category_Sci-Fi'),
 ('category_Sports', 'category_Sports'),
 ('category_Travel', 'category_Travel'),
 ('rating_NC-17', 'rating_NC-17'),
 ('rating_PG', 'rating_PG'),
 ('rating_PG-13', 'rating_PG-13'),
 ('rating_R', 'rating_R'),
 ('rental_duration_4', 'rental_duration_4'),
 ('rental_duration_5', 'rental_duration_5'),
 ('rental_duration_6', 'rental_duration_6'),
 ('rental_duration_7', 'rental_duration_7'),
 ('rental_rate_2.99', 'rental_rate_2.99'),
 ('rental_rate_4.99', 

,category_Animation,category_Children,category_Classics,category_Comedy,category_Documentary,category_Drama,category_Family,category_Foreign,category_Games,category_Horror,...,rating_NC-17,rating_PG,rating_PG-13,rating_R,rental_duration_4,rental_duration_5,rental_duration_6,rental_duration_7,rental_rate_2.99,rental_rate_4.99
468,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
332,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
1435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
380,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
99,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [19]:
X_test_transformed = np.concatenate([X_test_norm, X_test_cat], axis=1)

In [20]:
# Now we can make predictions on the test set:
predictions = classification.predict(X_test_transformed)
classification.score(X_test_transformed, y_test)

0.9747634069400631

In [21]:
y_test.value_counts()

False    202
True     115
Name: may, dtype: int64

In [22]:
pd.Series(predictions).value_counts()

False    194
True     123
dtype: int64

In [23]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[194,   8],
       [  0, 115]])

In [24]:
# bonus: KNN classifier: look at nearest neighbours and use the majority to determine class
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors=3, weights='uniform')
clf.fit(X_train_transformed, y_train)
predictions_clf = clf.predict(X_test_transformed)
clf.score(X_test_transformed, y_test)

0.5930599369085173

In [25]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[194,   8],
       [  0, 115]])